In [1]:
#Basic stuff
import logging
import os
from typing import Dict, Iterator
import time, datetime
from os import listdir
from os.path import isfile, join
import re
import pickle
import requests

#Add ./perceiver to path
import sys
sys.path.append('./perceiver')

#Math stuff
import numpy as np
import pandas as pd
import torch as t
from torch import optim

#Custom stuff
from utils import Meta
from sampler import get_train_batch, get_test_batch

#Model
from nbeats_perceiver import generic
from ranger import Ranger
from flatplusanneal import FlatplusAnneal
from losses import smape_2_loss, mape_loss, mase_loss, rmsse_loss

#Import tensorboard
from torch.utils.tensorboard import SummaryWriter

#Remove warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = NumbaPendingDeprecationWarning)

/home/eva01/anaconda3/envs/pytorch_2.0_full/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# URL of the file to download
file_url = 'https://osf.io/2pf93/download'

# send a GET request to the URL
response = requests.get(file_url)

# check if the request was successful (HTTP status code 200)
if response.status_code == 200:
    # open the file in binary write mode and write the response content to it
    with open('./data/binance_dataset_original_20220112.pkl', 'wb') as f:
        f.write(response.content)

# load the downloaded data from the file
data = pickle.load(open('./data/binance_dataset_original_20220112.pkl', 'rb'))

In [2]:
class HyperParameters():
    def __init__(self):
        #Meta date data
        self.date_array_len = 8 
        
        #Frequency
        self.seasonal_pattern = seasonal_pattern
        self.lookback = lookback
        self.outsample_size = Meta.horizons_map[self.seasonal_pattern]
        self.insample_size = self.lookback * self.outsample_size
        self.input_size = self.insample_size + self.date_array_len
        self.timeseries_frequency = Meta.frequency_map[self.seasonal_pattern]
        
        #Training params
        self.epochs = 5500 
        self.retrain_epochs = 9000
        self.batch_size = 32 
        self.lr_decay_step = self.epochs // 3
        self.learning_rate =  0.001

        #Options
        self.loss_options = ['MASE', 'SMAPE', 'MAPE']
        
        #stack params: input_size, output_size, stacks, layers, layer_size
        self.stacks = 20 
        self.layers = 4
        self.layer_size = 512
        self.stack_params = [self.input_size, self.outsample_size, self.stacks, self.layers, self.layer_size]

In [3]:
class Trainer(HyperParameters):
    def __init__(self):
        super(Trainer, self).__init__()
        self.seed = seed
        self.trainer_epoch = 0
        
        #Load data
        self.data = data 
        
        #Init model
        self.init_model()
    
    #Model parellelism
    def init_model(self):
        self.model = generic(self.stack_params)
        if t.cuda.device_count() > 1:
            self.model = t.nn.parallel.DataParallel(self.model, device_ids=list(range(t.cuda.device_count())))
        self.model.to("cuda:0")
    
    #Transform in tensor
    def to_tensor(self, array):
        return t.tensor(array, dtype=t.float32).to("cuda:0")
        
    #Loss step
    def loss_step(self, batch):
        x, x_mask, y, y_mask = map(self.to_tensor, batch)
        y_hat  = self.model(x, x_mask)
        loss = self.loss_func(x, self.timeseries_frequency, y_hat, y, y_mask)
        return loss
    
    #Possible loss functions
    def loss_fn(self, loss_name):
        def loss(x, freq, forecast, target, target_mask):
            if loss_name == 'MAPE':
                return mape_loss(forecast, target, target_mask)
            elif loss_name == 'MASE':
                return mase_loss(x, freq, forecast, target, target_mask)
            elif loss_name == 'SMAPE':
                return smape_2_loss(forecast, target, target_mask)
            else:
                raise Exception(f'Unknown loss function: {loss_name}')
        return loss
    
    #Batch functions
    def train_batch(self):
        #get_train_batch for self.data
        return get_train_batch(self.data['dates'], self.data['dates_array'], self.data['train_cutoff'],
                                 self.data['series'],
                                 self.data['start_offsets'], self.data['finish_offsets'],
                                 self.data['train_eligible'], self.data['compatibility'],
                                 self.insample_size, self.outsample_size, self.batch_size)
    
    def test_batch(self):
        return get_test_batch(self.data['dates'], self.data['dates_array'], self.data['train_cutoff'],
                                    self.data['series'],
                                    self.data['start_offsets'], self.data['finish_offsets'],
                                    self.data['test_eligible'], self.data['compatibility'],
                                    self.insample_size, self.outsample_size, self.batch_size)
    
    #Train
    def train(self):
        self.model.train()
        batch = self.train_batch()
        loss = self.loss_step(batch)
        loss.backward()
        self.train_loss = loss.detach()
        t.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
        self.optimizer.step()
        self.optimizer.zero_grad()
        self.writer.add_scalar('train_' + self.loss_name, self.train_loss, self.epoch)
    
    #Test
    def test(self):
        self.model.eval()
        with t.no_grad():
            batch = self.test_batch()
            x, x_mask, y, y_mask = map(self.to_tensor, batch)
            y_hat = self.model(x, x_mask)
            self.smape = self.loss_fn('SMAPE')(x, self.timeseries_frequency, y_hat, y, y_mask).cpu().numpy()
            self.mase = self.loss_fn('MASE')(x, self.timeseries_frequency, y_hat, y, y_mask).cpu().numpy()
            self.mape = self.loss_fn('MAPE')(x, self.timeseries_frequency, y_hat, y, y_mask).cpu().numpy()
            
    #Save checkpoint
    def save_checkpoint(self):
        try:
            t.save(self.model.module.state_dict(), './model_checkpoints/' + self.run_name + '.pt')
        except:
            t.save(self.model.state_dict(), './model_checkpoints/' + self.run_name + '.pt')
    
    #Load checkpoint
    def load_checkpoint(self, run_name):
        load = t.load('./model_checkpoints/' + run_name + '.pt')
        try:
            self.model.load_state_dict(load)
        except:
            self.model.module.load_state_dict(load)
        
    def log(self):
        self.save_checkpoint()
        self.writer.add_scalar('SMAPE', self.smape, self.epoch)
        self.writer.add_scalar('MASE', self.mase, self.epoch)
        self.writer.add_scalar('MAPE', self.mape, self.epoch)
    
    #Trainer steps
    def fit(self, meta_name, seed, loss_name, retrain = False):
        
        #Initial params
        self.meta_name = meta_name
        self.seed = seed
        self.loss_name = loss_name
        
        #Start time counter
        self.total_t0 = time.time()
        
        #Setup tensorflow logs
        self.run_name = '{}_{}_s{}_l{}_l{}'.format(self.meta_name, self.seasonal_pattern,
                                                   str(self.seed), self.lookback, self.loss_name)
        self.writer = SummaryWriter('./logs/{}'.format(self.run_name))

        #Setup optimizer, earning rate scheduler and loss function
        self.optimizer = Ranger(self.model.parameters(), lr = 1e-3)
        if not retrain:
            self.sched = FlatplusAnneal(self.optimizer, max_iter=self.epochs, step_size=0.7)
            epochs = self.epochs
            print('Training...')
        else:
            self.sched = FlatplusAnneal(self.optimizer, max_iter=self.retrain_epochs, step_size=0)
            epochs = self.retrain_epochs
            print('Retraining...')
        self.loss_func = self.loss_fn(self.loss_name)

        print('Run name:', self.run_name)

        for epoch in range(epochs):
            self.trainer_epoch += 1
            self.epoch = epoch
            t0 = time.time()
            self.train()
            t1 = time.time()
            ttime = str(datetime.timedelta(seconds = round(t1 - self.total_t0)))
            etime = str(datetime.timedelta(seconds = round(t1 - t0)))
            if epoch % 250 == 0:
                self.ttime = ttime
                self.test()
                self.log()
                # print("Epoch: {}\tSMAPE: {:.3f}\tMASE: {:.3f}\tMAPE: {:.3f}\tETime: {}\tTTime: {}"
                 # .format(epoch + 1, self.smape, self.mase, self.mape, etime, ttime))
            self.sched.step()

        self.ttime = ttime
        print('TTime:', ttime)
        
    def retrain(self, run_name, meta_name, seed, loss_name):
        self.load_checkpoint(run_name)
        self.fit(meta_name, seed, loss_name, retrain = True)

In [4]:
#Get todays date
today = datetime.date.today()

#Transform it into a string
today = str(today)

#function to produce the combinations
def get_combinations(name, n):
    tracking = {}
    combinations = []
    lookbacks = [2, 3, 4, 5, 6, 7]
    losses = ['MAPE', 'MASE', 'SMAPE']

    #Get all combinations of lookbacks and losses
    for lookback in lookbacks:
        for loss in losses:
            tracking[str(lookback) + '_' + loss] = {'counts':0}
            for i in range(n):
                combinations.append([name, lookback, loss, i])

    return tracking, combinations 

tracking, combinations = get_combinations('perc', 3)

len(combinations)

In [7]:
seasonal_pattern = 'Hourly'

#For experiments in to_train
for experiment in combinations:
    meta_name = experiment[0]
    lookback = experiment[1]
    loss_name = experiment[2]
    count = experiment[3]

    print('*************** Start time:', datetime.datetime.now().strftime("%H:%M:%S"))
    t.cuda.empty_cache()
    seed = np.random.randint(2**16)
    t.manual_seed(seed)
    trainer = Trainer()
    trainer.fit(meta_name, seed, loss_name)

    #update tracking
    tracking[str(lookback) + '_' + loss_name]['counts'] += 1

*************** Start time: 08:40:51
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Training...
Run name: perc_Hourly_s6412_l6_lMAPE


/home/eva01/TRADING_BOT/forecasting_model/model/ranger.py:115: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at /opt/conda/conda-bld/pytorch_1669968767843/work/torch/csrc/utils/python_arg_parser.cpp:1488.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


TTime: 2:18:03
*************** Start time: 10:58:57
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Training...
Run name: perc_Hourly_s17271_l6_lMAPE
TTime: 2:20:08
*************** Start time: 13:19:07
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Training...
Run name: perc_Hourly_s32776_l6_lMAPE
TTime: 2:19:49
*************** Start time: 15:38:58
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Training...
Run name: perc_Hourly_s34927_l6_lSMAPE
